In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from gpt_mini.model import GPT
from gpt_mini.trainer import Trainer
from gpt_mini.bpe import MidiDataset
import torch
from gpt_mini.config import DEFAULT_DEVICE, CONFIG

In [3]:
model_config = GPT.get_default_config()
# either model_type or (n_layer, n_head, n_embd) must be given in the config
model_config.model_type = None # 'gpt-nano'
model_config.n_layer = CONFIG["model"]["n_layer"]
model_config.n_head = CONFIG["model"]["n_head"]
model_config.n_embd = CONFIG["model"]["n_embed"]
# model_config.vocab_size = 50257 # 65535        # max number of vocabulary
model_config.vocab_size = CONFIG["model"]["vocab_size"]
# model_config.block_size = 256                  # input context length
model_config.block_size = CONFIG["model"]["block_size"]

model = GPT(model_config)

number of parameters: 2.51M


In [4]:
train_dataset = MidiDataset(
    CONFIG["preprocess"]["new_dataset_index"],
    CONFIG["tokenizer"]["model"],
    CONFIG["model"]["block_size"])
print(train_dataset)

/Users/robrohan/miniconda3/envs/comp838/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# your subclass of torch.utils.data.Dataset that emits example
# torch LongTensor of lengths up to 1024, with integers from [0,50257)
train_config = Trainer.get_default_config()
train_config.learning_rate = CONFIG["training"]["learning_rate"]
train_config.max_iters = CONFIG["training"]["max_iters"]
train_config.batch_size = CONFIG["training"]["batch_size"]
train_config.num_workers = CONFIG["training"]["workers"]
train_config.device = DEFAULT_DEVICE
trainer = Trainer(train_config, model, train_dataset)

running on device mps


In [6]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
        # Save checkpoints
        torch.save(
            trainer.model.state_dict(),
            f"./checkpoints/gpt_{trainer.iter_num}.pt")

trainer.set_callback('on_batch_end', batch_end_callback)
trainer.run()

/Users/robrohan/miniconda3/envs/comp838/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used

iter_dt 0.00ms; iter 0: train loss 10.85800
iter_dt 1140.40ms; iter 100: train loss 7.22034
iter_dt 1007.60ms; iter 200: train loss 7.41551
iter_dt 1009.08ms; iter 300: train loss 6.20288
iter_dt 995.15ms; iter 400: train loss 6.55572
iter_dt 1012.83ms; iter 500: train loss 6.24002
iter_dt 1005.59ms; iter 600: train loss 5.83739
iter_dt 1009.72ms; iter 700: train loss 5.52698
iter_dt 1008.25ms; iter 800: train loss 5.04322
iter_dt 1010.18ms; iter 900: train loss 5.26122
iter_dt 1007.90ms; iter 1000: train loss 5.43129
iter_dt 1012.55ms; iter 1100: train loss 4.55850
iter_dt 1007.60ms; iter 1200: train loss 4.42262
iter_dt 1007.78ms; iter 1300: train loss 4.54551
iter_dt 1201.49ms; iter 1400: train loss 4.43930
iter_dt 1035.96ms; iter 1500: train loss 4.19912
iter_dt 1027.37ms; iter 1600: train loss 4.19620
iter_dt 1006.98ms; iter 1700: train loss 4.12743
iter_dt 1024.13ms; iter 1800: train loss 3.76660
iter_dt 1019.40ms; iter 1900: train loss 4.19826
